<a href="https://colab.research.google.com/github/carol-furtado/Denoising-and-image-classification/blob/main/Image_Denoising_%26_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

In [2]:
from google.colab import files
uploaded = files.upload()


Saving train.zip to train.zip


In [3]:
import zipfile

with zipfile.ZipFile("train.zip", 'r') as zip_ref:
    zip_ref.extractall("train")


In [4]:
!ls /content


sample_data  train  train.zip


In [14]:
data_dir_clean = "/content/train/train/clean"
data_dir_noisy = "/content/train/train/noisy"
normalization_layer = tf.keras.layers.Rescaling(1./255)

clean = tf.keras.utils.image_dataset_from_directory(
    data_dir_clean,
    labels=None,

    image_size=(224,224),
    batch_size=32,
    shuffle=False
).map(lambda x: (normalization_layer(x)))


noisy = tf.keras.utils.image_dataset_from_directory(
    data_dir_noisy,
    labels=None,

    image_size=(224,224),
    batch_size=32,
    shuffle=False
).map(lambda x: (normalization_layer(x)))


Found 1795 files.
Found 1795 files.


In [16]:
import numpy as np

# Convert TF dataset to numpy array (no labels since label_mode=None)
def dataset_to_numpy(dataset):
    images = []
    for batch in dataset:
        images.append(batch.numpy())   # each batch is a tensor of shape (batch_size, h, w, c)
    return np.concatenate(images, axis=0)

# Example
clean_train_images = dataset_to_numpy(clean)
noisy_train_images = dataset_to_numpy(noisy)

print(clean_train_images.shape)
print(noisy_train_images.shape)


(1795, 224, 224, 3)
(1795, 224, 224, 3)


In [19]:
import tensorflow as tf
from tensorflow.keras import layers, Model

class ConvAutoencoder(Model):
    def __init__(self, latent_dim=128):
        super(ConvAutoencoder, self).__init__()


        self.encoder = tf.keras.Sequential([
            layers.Input(shape=(224, 224, 3)),
            layers.Conv2D(32, (3,3), activation='relu', padding='same', strides=2),
            layers.Conv2D(64, (3,3), activation='relu', padding='same', strides=2),
            layers.Conv2D(128, (3,3), activation='relu', padding='same', strides=2),
            layers.Conv2D(256, (3,3), activation='relu', padding='same', strides=2),
            layers.Flatten(),
            layers.Dense(latent_dim, activation='relu')
        ])


        self.decoder = tf.keras.Sequential([
            layers.Dense(14*14*256, activation='relu'),
            layers.Reshape((14, 14, 256)),
            layers.Conv2DTranspose(128, (3,3), strides=2, activation='relu', padding='same'),
            layers.Conv2DTranspose(64, (3,3), strides=2, activation='relu', padding='same'),
            layers.Conv2DTranspose(32, (3,3), strides=2, activation='relu', padding='same'),
            layers.Conv2DTranspose(3, (3,3), strides=2, activation='sigmoid', padding='same')
        ])

    def call(self, input_data):
        encoded = self.encoder(input_data)
        decoded = self.decoder(encoded)
        return decoded


In [20]:

import tensorflow.image as tfi

def psnr_metric(y_true, y_pred):
    return tfi.psnr(y_true, y_pred, max_val=1.0)

def ssim_metric(y_true, y_pred):
    return tfi.ssim(y_true, y_pred, max_val=1.0)


In [21]:

#compile
autoencoder=ConvAutoencoder()
autoencoder.compile(
    optimizer='adam',
    loss='mse',
    metrics=[psnr_metric, ssim_metric]
)


In [23]:
history = autoencoder.fit(
    x=noisy,
    y=clean,
    epochs=20,
    batch_size=32,
    validation_data=(noisy_train_images, clean_train_images)
)


ValueError: When providing `x` as a tf.data.Dataset, `y` should not be passed. Instead, the targets should be included as part of the tf.data.Dataset.